# Retroactive Data Analysis

## Imports

In [3]:
import os
import pandas
import statsmodels.api as sm
import sys
sys.path.append(os.path.dirname(os.getcwd()))

In [41]:
from mpow import load_data, plotting, regression
plotting.output_notebook(hide_banner=True)

## Load data

In [5]:
data = load_data.norm_daily_data()
details = load_data.norm_detail_data()

In [6]:
data.head()

,Patient,DayNum,Intake,PainScore,NumObs,AgeAtAdmit,Gender,ImpairmentGroup,Depression
0,1,1,80.0,29.0,7,32.0,Female,Spinal_Cord_Dysfunction,0.0
1,1,2,60.0,24.0,9,32.0,Female,Spinal_Cord_Dysfunction,0.0
2,1,3,70.0,38.0,11,32.0,Female,Spinal_Cord_Dysfunction,0.0
3,1,4,40.0,23.0,11,32.0,Female,Spinal_Cord_Dysfunction,0.0
4,2,1,75.0,63.0,14,56.0,Female,Spinal_Cord_Dysfunction,1.0


In [7]:
details.head()

,Patient,AgeAtAdmit,Gender,ImpairmentGroup,Depression
0,1,32,Female,Spinal_Cord_Dysfunction,0
1,2,56,Female,Spinal_Cord_Dysfunction,1
2,3,82,Male,Stroke,0
3,4,92,Male,Debility,0
4,5,67,Female,Orthopaedic_Disorders,1


## Explore details

### Age

Overall Distribution

In [9]:
plotting.show(plotting.histogram(details.AgeAtAdmit, bins=15))

Conditional on gender

In [16]:
details[['Gender', 'AgeAtAdmit']].groupby('Gender').median()

,AgeAtAdmit
Gender,
Female,67
Male,69


Conditional on depression

In [17]:
details[['Depression', 'AgeAtAdmit']].groupby('Depression').median()

,AgeAtAdmit
Depression,
0,68
1,67


Conditional on impairment

In [20]:
details[['ImpairmentGroup', 'AgeAtAdmit']].groupby('ImpairmentGroup').median().sort_values('AgeAtAdmit')

,AgeAtAdmit
ImpairmentGroup,
Burns,24.0
Congenital_Deformities,25.0
Amputations,58.0
Neurologic_conditions,58.0
Neuromuscular_disorders,60.5
Stroke,62.0
Brain_Dysfunction,63.0
Pulmonary_Disorders,64.0
Debility,68.0


### Impairment

In [23]:
details.ImpairmentGroup.value_counts()

Orthopaedic_Disorders      33
Spinal_Cord_Dysfunction    26
Stroke                     13
Debility                    5
Amputations                 5
Cardiac                     4
Neuromuscular_disorders     4
Pulmonary_Disorders         3
Neurologic_conditions       3
Major_Multiple_Trauma       2
Burns                       1
Congenital_Deformities      1
Brain_Dysfunction           1
Name: ImpairmentGroup, dtype: int64

## Depression Gender Interaction

Model: $$s_{t,i} \sim \beta_{0} + \beta_{1}p_{t,i} + \alpha^{df} + \alpha^{nf} + \alpha^{dm} + \alpha^{nm} + \epsilon_{t,i}$$

Sufficiency?

In [36]:
dgi_cats = {2:'FemNon',3:'MaleNon',4:'FemDep',6:'MaleDep'}
dgi_counts = (((details.Depression + 1) * ((details.Gender=='Male').astype(int) + 2))
            .to_frame('DepGenInter').applymap(dgi_cats.get))
dgi_counts.DepGenInter.value_counts()

MaleNon    35
FemNon     32
FemDep     28
MaleDep     6
Name: DepGenInter, dtype: int64

In [39]:
dgi = data.copy()
dgi['FemDep'] = ((dgi.Depression==1)&(dgi.Gender=='Female')).astype(int)
dgi['MaleDep'] = ((dgi.Depression==1)&(dgi.Gender=='Male')).astype(int)
dgi['FemNon'] = ((dgi.Depression==0)&(dgi.Gender=='Female')).astype(int)
dgi['MaleNon'] = ((dgi.Depression==0)&(dgi.Gender=='Male')).astype(int)

In [45]:
dgi_model = regression.ols(dgi.dropna(), ['PainScore', 'FemDep', 'MaleDep', 'FemNon', 'MaleNon'], 'Intake')
dgi_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Intake   R-squared:                       0.396
Model:                            OLS   Adj. R-squared:                  0.394
Method:                 Least Squares   F-statistic:                     164.4
Date:                Wed, 22 Nov 2017   Prob (F-statistic):          3.24e-108
Time:                        22:08:52   Log-Likelihood:                -4695.9
No. Observations:                1008   AIC:                             9402.
Df Residuals:                    1003   BIC:                             9426.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.8485      1.118      7.018      0.000       5.654      10.043
PainScore      1.0662      0.051     20.970      0.000       0.966       1.166
FemDep        16.2946      1.506     10.817      0.000      13.338      19.251
MaleDep        0.6792      2.597      0.262      0.794      -4.418       5.776
FemNon        -5.8837      1.477     -3.982      0.000      -8.783      -2.984
MaleNon       -3.2416      1.281     -2.531      0.012      -5.755      -0.728
==============================================================================
Omnibus:                      408.338   Durbin-Watson:                   0.504
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2052.160
Skew:                           1.825   Prob(JB):                         0.00
Kurtosis:                       8.961   Cond. No.                     1.03e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.61e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [46]:
-5.8837 + 1.477

-4.4067

In [47]:
-3.2416 - 1.281

-4.5226